# Consultas Integradas

Este notebook tiene el fin de generar las consultas planteadas en el trabajo práctico integrador

#### Importo librerias

In [3]:
import os
from pathlib import Path

import pandas as pd
from pymongo.errors import ConnectionFailure

from db_connections import client, db_neo4j, db_redis
from src import mongo, neo4j, utils, redis

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
# =====================
# TEST DE CONEXIONES
# =====================

#Conecto con MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Conecto con Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Conecto con Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

## A.Mostrar los usuarios que visitaron “Bariloche”. 

In [ ]:
query = """ 
MATCH (U:Usuario)-[:VISITO]->(D:Destino)
WHERE D.ciudad='Mendoza'
RETURN U.usuario_id AS id, U.nombre AS Nombre, U.apellido AS Apellido
"""

usuarios = neo4j.consulta(db_neo4j, query)
usuarios

### B.Mostrar los amigos de Juan que visitaron algún destino que visitó él, mostrar el nombre del Usuario y el destino. 

In [ ]:
persona = input("Introduce tu nombre para saber qué lugares visitaste con tus amigos: ")

query = f"""
MATCH (u:Usuario {{nombre: '{persona}'}})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (u)-[:VISITO]->(d:Destino)<-[:VISITO]-(amigo)
RETURN 
    amigo.nombre AS Nombre,
    collect(DISTINCT d.ciudad) AS Destinos_Compartidos
ORDER BY Nombre
"""
print("\n" + "-"*60)
print(f"AMIGOS DE {persona.upper()} QUE VISITARON LOS MISMOS DESTINOS")
print("-"*60 + "\n")

usuarios = neo4j.consulta(db_neo4j, query)
usuarios


###### C.Sugerir destinos a un usuario que no haya visitado él ni sus amigos. 

###### d. Recomendar destinos basados en viajes de amigos.

In [ ]:
persona = input("Introduce tu nombre para saber qué lugares te recomendamos ")

query = f"""
MATCH (u:Usuario {{nombre: '{persona}'}})-[:AMIGO_DE]-(amigo:Usuario)
MATCH (amigo)-[:VISITO]->(d:Destino)
RETURN DISTINCT d.ciudad AS Destino_Recomendado
ORDER BY d.ciudad
"""

print("\n" + "-"*60)
print(f"AMIGOS DE {persona.upper()} QUE VISITARON LOS MISMOS DESTINOS")
print("-"*60 + "\n")

Destinos = neo4j.consulta(db_neo4j, query)
Destinos


###### e. Listar los hoteles en los destinos recomendados del punto anterior. 

###### f.Ver las reservas en proceso, es decir que aún no están concretadas. 

###### g. Listar los usuarios conectados actualmente. 

###### h. Mostrar los destinos con precio inferior a $100.000

###### i. Mostrar todos los Hoteles de “Jujuy”.

###### j. Mostrar la cantidad de hoteles de un destino que guste. 

###### k. Mostrar las actividades de “Ushuaia” del tipo “aventura”.

###### l. Mostrar la cantidad de reservas concretadas de cada usuario. Mostrar el usuario y la cantidad 

#### Estadísticas

###### i.Destino más visitado

###### ii.Hotel más barato

###### iii.Actividad más popular.

#### Modificaciones

###### a.Incrementar el precio de las actividades de Tucuman en 5% 

###### b. Agregar al hotel id=1 el servicio de SPA 

###### c. Eliminar el destino que desee

###### d. Eliminar un usuario que desee 

###### e. Eliminar las relaciones AMIGO_DE para un usuario que quiera. 